In [ ]:
def main():

  # Выбор носителя
  device = 'cuda'

  # Установка библиотек
  ! pip install pytelegrambotapi
  ! pip install -U transformers
  ! pip install -U accelerate
  ! pip install -i https://pypi.org/simple/ bitsandbytes
  ! pip install sentence_transformers
  ! pip install python-dotenv


  # Загрузка бибилиотек
  import telebot
  from telebot import types
  import re
  import random
  import torch
  from transformers import AutoModelForCausalLM, AutoTokenizer


  # Подключение к Google Drive (для работы в Google Colab)
  from google.colab import drive
  drive.mount('/content/drive', force_remount=True)


  # Загрузка токена доступа к телеграм-боту
  from dotenv import load_dotenv
  import os
  load_dotenv('drive/MyDrive/Colab Notebooks/LLaMA/.env')
  TOKEN = os.getenv("TOKEN")


  model = AutoModelForCausalLM.from_pretrained("AndrewDOrlov/LLaMA-v2-merged-8b")
  tokenizer = AutoTokenizer.from_pretrained("AndrewDOrlov/LLaMA-v2-merged-8b", trust_remote_code=True)

  model.generation_config.eos_token_id=2799
  def get_answer(user_prompt):
    """
    функция для получения ответа на вопрос пользователя
    """
    runtimeFlag = "cuda:0"
    system_prompt = 'Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n'
    B_INST, E_INST = "### Instruction:\n", "### Response:\n"
    prompt = f"{system_prompt}{B_INST}{user_prompt.strip()}\n\n{E_INST}"
    inputs = tokenizer([prompt], return_tensors="pt").to(runtimeFlag)
    output = model.generate(**inputs, max_new_tokens=200)
    text = tokenizer.decode(output[0], skip_prompt=True, skip_special_tokens=True)
    result = re.split(r'\n', text)
    torch.cuda.empty_cache()
    return result[6]


  # Определение параметров телеграм-бота (telebot)
  your_bot = TOKEN
  bot = telebot.TeleBot(your_bot)

  # Готовые вопросы для телеграм-бота
  questions = [
    'Можешь рассказать о C++?',
    'Что такое Dark Web?',
    'Что такое Midjourney?',
    'Что такое Stable Diffusion?',
    'Что такое Netfilter?',
    'Что такое LLaMA?',
    'Что такое уязвимость нулевого дня?',
    'Хочу узнать про 5G',
    'Что такое Битрикс24?',
    'Что такое SOCKS?',
    'Что такое DALL-E?',
    'Что значит "эмулятор"?',
    'Что такое троян?',
    'Что такое интернет вещей?',
    'Что такое Pretty Good Privacy?',
    'Что такое макровирус?',
    'Что такое лутбокс?',
    'Какой хакер считается этичным?',
    'Что такое honeypot?',
    'Расскажи про первый компьютерный червь',
    'Расскажи про UNIX',
    'Что такое GitHub?',
    'Что такое Linux?',
    'Что такое ChatGPT?',
    'Расскажи про проект Apache',
    'Что такое резервная копия?',
    'Что такое Clang?',
    'Что такое биткойн?',
    'Что такое Internet Standard?',
    'Что значит Sensitive Information?'
  ]

  # Путь начала работы телеграм-бота
  @bot.message_handler(commands=["start"])
  def start(message, res=False):
    markup=types.ReplyKeyboardMarkup(resize_keyboard=True)
    item1=types.KeyboardButton("Случайный вопрос")
    item2=types.KeyboardButton("Помощь")
    markup.row(item1, item2)
    bot.send_message(message.chat.id, 'Здравствуйте! Я – Сиби (Cybersecurity_bot). Я буду рад ответить на ваши вопросы по кибербезопасности. Пожалуйста, введите ваш вопрос:', reply_markup=markup)

  # Путь телеграм-бота для получения и обработки вопроса пользователя к базе знаний
  @bot.message_handler(content_types=["text"])
  def handle_text(message):

    if message.text.strip() == 'Помощь':
      answer = 'Введите интересующий вас вопрос по теме кибербезопасности или нажмите кнопку "Случайный вопрос". Я дам краткий ответ и подберу подходящую статью из базы знаний. Лучше всего я понимаю вопросы вида "Что такое ChatGPT?". Если ответ не подошел, попробуйте сформулировать вопрос по-другому.'
      bot.send_message(message.chat.id, answer)

    elif message.text.strip() == 'Случайный вопрос':
      question = questions[random.randint(0, len(questions) - 1)]
      bot.send_message(message.chat.id, question)
      answer = get_answer(question)
      bot.send_message(message.chat.id, "Ответ: " + answer)

    else:
      question = message.text.strip()
      answer = get_answer(question)
      bot.send_message(message.chat.id, "Ответ: " + answer)


  print("loaded")
  # активация телеграм-бота
  torch.cuda.empty_cache()
  bot.polling(none_stop=True, interval=0)


if __name__ == "__main__":
    main()

Looking in indexes: https://pypi.org/simple/
Mounted at /content/drive


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

loaded
